In [31]:
import sys
sys.path.append("..") 

In [32]:
# import dicom2nifti
# import dicom2nifti.settings as settings
# settings.disable_validate_orthogonal()
# settings.enable_resampling()
# settings.set_resample_spline_interpolation_order(1)
# settings.set_resample_padding(-1000)
# settings.disable_validate_orientation()
# dicom2nifti.convert_directory(r'G:\Projects and Work\Mouse Heart Segmentation\Mice_CT_Dataset\volumes\C57-#6-MicroCT-020421 - Cardiac CT_110734-systole',
#                             'abc',reorient=False)

In [33]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import os
import torch.cuda
import albumentations as albu
import segmentation_models_pytorch as smp
from  segmentation_models_pytorch.utils.base import Metric
from segmentation_models_pytorch.base.modules import Activation
from collections import defaultdict
from torch.utils.tensorboard import SummaryWriter
import torchvision

In [34]:
from Utils.dataset_utils import *
from Utils.visualization_utils import *

In [35]:
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [36]:
DATASET_2D_BASE_PATH=Path('G:\Projects and Work\Mouse Heart Segmentation\Mice_CT_Dataset\Axials')
WIDTH=320
HEIGHT=320

In [37]:


def get_training_augmentation():
    train_transform = [

        albu.Resize(HEIGHT, WIDTH),
        albu.ShiftScaleRotate(scale_limit=0.20, rotate_limit=30, shift_limit=0.1, p=1, border_mode=cv2.BORDER_CONSTANT),
        albu.RandomCrop(height=320, width=320),
        albu.Blur(blur_limit=3, p=0.4),
        albu.GaussNoise(p=0.5),
        albu.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        albu.RandomBrightness(p=0.75)

    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    test_transform = [
        albu.Resize(512, 512)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [38]:
df_train=load_data(DATASET_2D_BASE_PATH/'train')[0:20]
df_val=load_data(DATASET_2D_BASE_PATH/'val')[0:20]
df_train.head()

,images,masks
0,G:\Projects and Work\Mouse Heart Segmentation\...,G:\Projects and Work\Mouse Heart Segmentation\...
1,G:\Projects and Work\Mouse Heart Segmentation\...,G:\Projects and Work\Mouse Heart Segmentation\...
2,G:\Projects and Work\Mouse Heart Segmentation\...,G:\Projects and Work\Mouse Heart Segmentation\...
3,G:\Projects and Work\Mouse Heart Segmentation\...,G:\Projects and Work\Mouse Heart Segmentation\...
4,G:\Projects and Work\Mouse Heart Segmentation\...,G:\Projects and Work\Mouse Heart Segmentation\...


### Training

In [39]:
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'
TRAIN_RUNS_PATH=r'G:\Projects and Work\Mouse Heart Segmentation\runs'
MODEL_NAME='Unet'
BATCH_SIZE=2
EPOCHS=3


In [43]:
HUMAN_WEIGHTS_PATH=r'G:\Projects and Work\Mouse Heart Segmentation\Trained Weights - Human Heart\imagenet'

In [41]:
ENCODERS = ['se_resnext50_32x4d','resnet101','efficientnet-b4']
# ENCODERS = ['se_resnext50_32x4d']

In [42]:
for j in range(len(ENCODERS)):
    
#     model = smp.Unet(
#     encoder_name=ENCODERS[j], 
#     encoder_weights=ENCODER_WEIGHTS, 
#     classes=1, 
#     activation=ACTIVATION,
#     )
  

    model=torch.load(os.path.join(HUMAN_WEIGHTS_PATH,f'Unet_{ENCODERS[j]}','best.pt'))
    preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODERS[j], ENCODER_WEIGHTS)
    
    WEIGHTS_PATH=os.path.join(TRAIN_RUNS_PATH,f'{MODEL_NAME}_{ENCODERS[j]}')
    if os.path.exists(WEIGHTS_PATH)==False:
        os.mkdir(WEIGHTS_PATH)
    else:
        print(f"Warning! Directory {WEIGHTS_PATH } already exists")
    
    train_dataset = Dataset(
    df_train['images'], 
    df_train['masks'], 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn)
    )

    valid_dataset = Dataset(
        df_val['images'], 
        df_val['masks'], 
        augmentation=get_validation_augmentation(), 
        preprocessing=get_preprocessing(preprocessing_fn)
    ) 

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    loss = smp.utils.losses.DiceLoss()
    metrics = [
        smp.utils.metrics.IoU(threshold=0.5),
        smp.utils.metrics.Fscore()

    ]

    optimizer = torch.optim.Adam([ 
        dict(params=model.parameters(), lr=0.0001),
    ])
    train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
    )

    valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
    )
    min_loss = 100000000
    train_history=defaultdict(list)
    valid_history=defaultdict(list)
    
    comment = f'model = {ENCODERS[j]}'
    tb = SummaryWriter(comment)
#     images, labels = next(iter(train_loader))
#     grid = torchvision.utils.make_grid(images)
#     images=images.to(DEVICE)
#     tb.add_image("images", grid)
#     tb.add_graph(model, images)
#     tb.close()
    for i in range(0, EPOCHS):

        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(valid_loader)

        # do something (save model, change lr, etc.)
        if min_loss > valid_logs['dice_loss']:
            min_loss = valid_logs['dice_loss']
            torch.save(model, os.path.join(WEIGHTS_PATH,f'best_{str(i)}_{round(min_loss,4)}.pt'))
            print('Model saved!')

        if i == 25:
            optimizer.param_groups[0]['lr'] = 1e-5
            print('Decrease decoder learning rate to 1e-5!')
        # Maintain History
        for log_key in train_logs.keys():
            train_history[log_key].append(train_logs[log_key])
            valid_history[log_key].append(valid_logs[log_key])
            tb.add_scalar(f'train_{log_key}',train_logs[log_key], i)
            tb.add_scalar(f'validation_{log_key}',valid_logs[log_key], i)
        
#     tb.add_hparams(
#         {"model": ENCODERS[j]},{"min_loss": min_loss}
#     )
     
    del model
    
    pd.DataFrame(valid_history).to_csv(os.path.join(WEIGHTS_PATH,'validation_logs.csv'))
    pd.DataFrame(train_history).to_csv(os.path.join(WEIGHTS_PATH,'train_logs.csv'))
    print(f"{WEIGHTS_PATH} Completed!!!")
tb.close()

FileNotFoundError: [Errno 2] No such file or directory: 'G:\\Projects and Work\\Mouse Heart Segmentation\\Trained Weights - Human Heart\\Unet_se_resnext50_32x4d\\best.pt'